<a href="https://www.kaggle.com/code/pranayrishith16/spaceship-titanic-notebook-acheived-80-accuracy?scriptVersionId=134722647" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Data Preprocessing
Importing all the libraries needed for preprocessing

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import seaborn as sns
%matplotlib inline
sns.set_style('white')


In [ ]:
data = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')

In [ ]:
data_t = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')

Copying data into another variable inorder to makeup authenticity of the original variable.

Just kidding. Copying data is better if experimenting.

In [ ]:
data1 = data.copy(deep=False)
data1

dropping passengerID. We dont have any usecase with it.

In [ ]:
data1.set_index('PassengerId',inplace=True)

getting all info on the data. gives an idea on the data like column names, number of not null values and datatype of the features.

In [ ]:
data1.info()

getting a description on the features. This is mainly on countinuous variables


In [ ]:
data1.describe()

Specific datatypes as categorical features dont have min and max

In [ ]:
data1.describe(include=('object','bool'))

Finding how many null values in the dataset

In [ ]:
data1.isna().sum()

As there are many categorical features we need to find all the values in it. Categorical values are limited.

value_counts() is used in order to return how many instances of that value has been present.

In [ ]:
cols = data1.columns
for col in cols:
    if data1[col].dtype != 'float64':
        print(data1[col].value_counts())

Now lets work on missing values.
As there are few missing values lets try by replacing them with some values like mean, median or mode.

**RoomService**, **FoodCourt**, **ShoppingMall, VrDeck and Spa** are float values. when observing their mean values and min, max values in the describe function, we can observe than more than 50% of people have not spent on these services. So I have opted to use median to fill null values.

Before filling **cryosleep and vip** features, these featuers have true and false as values which means these are boolean, but the datatype is object, so first changing the datatype

Next **Homeplanet, destination**, most people homeplanet is earth so choosing earth as  null value(you can find in above cell) similar trend goes to destination

**Cryosleep and VIP** are boolean so filling False as the missing value.
as **VIP and cryosleep** most passengers are having False so I have chose it.

**Age** is a countinuous variable by observing the trend of age we can comfortably settle for mean.

In [ ]:
data1['RoomService'] = data1['RoomService'].fillna(data1['RoomService'].median())
data1['FoodCourt'] = data1['FoodCourt'].fillna(data1['FoodCourt'].median())
data1['ShoppingMall'] = data1['ShoppingMall'].fillna(data1['ShoppingMall'].median())
data1['VRDeck'] = data1['VRDeck'].fillna(data1['VRDeck'].median())
data1['Spa'] = data1['Spa'].fillna(data1['Spa'].median())


data1['CryoSleep'] = data1['CryoSleep'].astype(bool)
data1['VIP'] = data1['VIP'].astype(bool)



data1['HomePlanet'] = data1['HomePlanet'].fillna('Earth')
data1['Destination'] = data1['Destination'].fillna('TRAPPIST-1e')


data1['CryoSleep'] = data1['CryoSleep'].fillna(False)
data1['VIP'] = data1['VIP'].fillna(False)


data1['Age'] = data1['Age'].fillna(data1['Age'].mean())

data1['Cabin'] = data1['Cabin'].fillna('B/0/S')

**RoomService, FoodCourt, ShoppingMall, VrDeck, Spa** are collectively expenses so creating another feature calling **total_spending**

In [ ]:
data1['Total_spending'] = data1['RoomService'] + data1['FoodCourt'] + data1['ShoppingMall'] + data1['VRDeck'] + data1['Spa']

**Cabin** is composed of  deck/number/side so creating new columns as **cabin_side**** and **cabin_deck** consisting as deck and side. and dropping cabin column for now.

In [ ]:
data1['Cabin_Side'] = data1['Cabin'].str.split("/").str[2]
data1['Cabin_Deck'] = data1['Cabin'].str.split("/").str[0]
data1 = data1.drop('Cabin',axis=1)

Dropping all other missing values (for now)

In [ ]:
data1 = data1.dropna()
data1.drop("Name",axis="columns",inplace=True)

In [ ]:
data1.columns

# Data visualization




**Categorical**: HomePlanet, CryoSleep, Destination, VIP, Transported, Cabin_Side, Cabin_Deck

**Numerical**: Age, RoomService, FoodCourt, ShoppingMall, Spa, VRDeck, Total_spending

In [ ]:
sns.barplot(x='Transported',y='HomePlanet',data=data1)

In [ ]:
sns.barplot(x='Transported',y='Destination',data=data1)

In [ ]:
sns.barplot(x='Transported',y='CryoSleep',data=data1)

In [ ]:
sns.barplot(x='Transported',y='VIP',data=data1)

In [ ]:
sns.barplot(x='Transported',y='Cabin_Deck',data=data1)

In [ ]:
sns.barplot(x='Transported',y='Cabin_Side',data=data1)

In [ ]:
data1['CryoSleep'].replace({False:0,True:1},inplace=True)
data1['Transported'].replace({False:0,True:1},inplace=True)
data1['VIP'].replace({False:0,True:1},inplace=True)

In [ ]:
f = plt.figure(figsize=(24,20))
sns.heatmap(data1.corr(),annot=True,vmin=0,vmax=1)

In [ ]:
data1['HomePlanet'] = data1['HomePlanet'].astype('category')
data1['Destination'] = data1['Destination'].astype('category')
data1['Cabin_Deck'] = data1['Cabin_Deck'].astype('category')
data1['Cabin_Side'] = data1['Cabin_Side'].astype('category')

In [ ]:
data1 = pd.get_dummies(data1,columns=['HomePlanet','Destination','Cabin_Deck','Cabin_Side'])
data1.columns

In [ ]:
f = plt.figure(figsize=(24,20))
sns.heatmap(data1.corr(),annot=True,vmin=0,vmax=1)

In [ ]:
Y = data1['Transported']
X = data1.drop('Transported',axis='columns')

# Model Training

Tried various models

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import metrics

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import (RandomForestClassifier, 
                              AdaBoostClassifier, 
                              BaggingClassifier, 
                              GradientBoostingClassifier,
                              HistGradientBoostingClassifier
                             )
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import (BernoulliNB,
                                 GaussianNB,
                                 MultinomialNB
                                )

import catboost as ctb


In [ ]:
X_train,X_test, y_train, y_test = train_test_split(X,Y,test_size=0.3,random_state=41)

In [ ]:
rfc = RandomForestClassifier(n_estimators=1000)
rfc.fit(X_train,y_train)
rfc_pred = rfc.predict(X_test)
print("Random forest accuracy: {}".format(metrics.accuracy_score(y_test,rfc_pred)))
print()
print()


abc = AdaBoostClassifier()
abc.fit(X_train,y_train)
abc_pred = abc.predict(X_test)
print("Adaboost accuracy: {}".format(metrics.accuracy_score(y_test,abc_pred)))
print()
print()


bag = BaggingClassifier()
bag.fit(X_train,y_train)
bag_pred = bag.predict(X_test)
print("Bagging accuracy: {}".format(metrics.accuracy_score(y_test,bag_pred)))
print()
print()


hgbc = HistGradientBoostingClassifier()
hgbc.fit(X_train,y_train)
hgbc_pred = hgbc.predict(X_test)
print("Hist gradient boosting accuracy: {}".format(metrics.accuracy_score(y_test,hgbc_pred)))
print()
print()


gnb = GaussianNB()
gnb.fit(X_train,y_train)
gnb_pred = gnb.predict(X_test)
print("Gaussian NB accuracy: {}".format(metrics.accuracy_score(y_test,gnb_pred)))
print()
print()


mnb = MultinomialNB()
mnb.fit(X_train,y_train)
mnb_pred = mnb.predict(X_test)
print("Multinomial NB accuracy: {}".format(metrics.accuracy_score(y_test,mnb_pred)))
print()
print()


CBC = ctb.CatBoostClassifier(silent=True,
                             depth=6,
                             iterations=300,
                            )
CBC.fit(X_train, y_train)
CBC_pred = CBC.predict(X_test)
print("CBC accuracy: {}".format(metrics.accuracy_score(y_test,CBC_pred)))

Preprocessing test data set and predicting output.

In [ ]:
data_t.set_index('PassengerId',inplace=True)

data_t['RoomService'] = data_t['RoomService'].fillna(data_t['RoomService'].median())
data_t['FoodCourt'] = data_t['FoodCourt'].fillna(data_t['FoodCourt'].median())
data_t['ShoppingMall'] = data_t['ShoppingMall'].fillna(data_t['ShoppingMall'].median())
data_t['VRDeck'] = data_t['VRDeck'].fillna(data_t['VRDeck'].median())
data_t['Spa'] = data_t['Spa'].fillna(data_t['Spa'].median())


data_t['CryoSleep'] = data_t['CryoSleep'].astype(bool)
data_t['VIP'] = data_t['VIP'].astype(bool)



data_t['HomePlanet'] = data_t['HomePlanet'].fillna('Earth')
data_t['Destination'] = data_t['Destination'].fillna('TRAPPIST-1e')


data_t['CryoSleep'] = data_t['CryoSleep'].fillna(False)
data_t['VIP'] = data_t['VIP'].fillna(False)


data_t['Age'] = data_t['Age'].fillna(data_t['Age'].mean())
data_t['Cabin'] = data_t['Cabin'].fillna('B/0/S')

In [ ]:
data_t['Total_spending'] = data_t['RoomService'] + data_t['FoodCourt'] + data_t['ShoppingMall'] + data_t['VRDeck'] + data_t['Spa']
data_t['Cabin_Side'] = data_t['Cabin'].str.split("/").str[2]
data_t['Cabin_Deck'] = data_t['Cabin'].str.split("/").str[0]
data_t = data_t.drop('Cabin',axis=1)
data_t.drop("Name",axis="columns",inplace=True)

data_t['CryoSleep'].replace({False:0,True:1},inplace=True)
data_t['VIP'].replace({False:0,True:1},inplace=True)

data_t['HomePlanet'] = data_t['HomePlanet'].astype('category')
data_t['Destination'] = data_t['Destination'].astype('category')
data_t['Cabin_Deck'] = data_t['Cabin_Deck'].astype('category')
data_t['Cabin_Side'] = data_t['Cabin_Side'].astype('category')

data_t = pd.get_dummies(data_t, columns=['HomePlanet','Destination','Cabin_Deck','Cabin_Side'])
data_t.columns


In [ ]:
y_pred = CBC.predict(data_t)

In [ ]:
sub = pd.DataFrame({'Transported':y_pred.astype(bool)},index=data_t.index)
sub.head()

In [ ]:
sub.to_csv('submission.csv')